# Inviz Example

Inviz was made for exploring cosmological theories. It has the ability to interface with whatever version of the CLASS cosmology code you have installed to compute observables based on input data, and compare them to the $\Lambda$CDM model. In this notebook, we're going to be exploring a set of data that represents a theory of interacting dark matter (IDM). They can be found in the data/test_IDM_n_0 folder on GitHub.

*Note*: this notebook uses the latest release of inviz on pypi.org. Things like function arguments and required data preprocessing might not match up with the latest version on GitHub.

In [ ]:
from inviz import *
hv.extension('bokeh')
pn.extension()

The files in the test_IDM_n_0 folder are the following:
- The .paramnames file, which is a list of the cosmological parameters spanned by the chains
- The chains themselves, which are .txt files and contain values for each parameter 

Inviz requires Pandas Dataframes as input. We can use inviz's built-in functions for reading in these files and construct a dataframe:


In [ ]:
# load in the data
loglkl = ['loglkl']
param_names, latex_params = load_params('../data/test_IDM_n_0/2022-05-04_75000_.paramnames')
params_latex_form = dict(zip(param_names, latex_params))
column_names = loglkl + param_names
df = pd.DataFrame(columns=column_names)
for i in trange(1,57):
    temp = load_data('../data/test_IDM_n_0/2022-05-04_75000__{}.txt'.format(i), column_names=column_names)
    df = pd.concat([df,temp]).reset_index(drop=True)

Now, if we want to use CLASS to calculate the observed power spectra resulting from our custom cosmology, we need to modify the chains by removing the nuisance parameters and adding additional CLASS-specific parameters. 

In [ ]:
# prepare your data for CLASS computation. this step is specific to your dataset
# remove nuisance parameters
cosmo_df = df.drop(columns=['loglkl', 'z_reio', 'A_s', 'sigma8', '100theta_s', 'A_cib_217', 'xi_sz_cib', 'A_sz', 'ps_A_100_100', 'ps_A_143_143', 'ps_A_143_217', 'ps_A_217_217', 'ksz_norm', 
                 'gal545_A_100', 'gal545_A_143', 'gal545_A_143_217', 'gal545_A_217', 'galf_TE_A_100', 'galf_TE_A_100_143', 'galf_TE_A_100_217', 'galf_TE_A_143', 'galf_TE_A_143_217', 
                 'galf_TE_A_217', 'calib_100T', 'calib_217T', 'A_planck'])
cosmo_df['omega_b'] = df['omega_b'] * 1e-2
cosmo_df['sigma_dmeff'] = df['sigma_dmeff'] * 1e-25
cosmo_df = cosmo_df.rename(columns={'H0':'h'})
cosmo_df['h'] = cosmo_df['h'] * 1e-2
cosmo_df['omega_cdm'] = 1e-15
cosmo_df['npow_dmeff'] = 0.0
cosmo_df['Vrel_dmeff'] = 0.0
cosmo_df['dmeff_target'] = 'baryons'
cosmo_df['m_dmeff'] = 1e-3

# format for CDM version
cosmoCDM_df = cosmo_df.drop(columns=['sigma_dmeff', 'omega_cdm', 'npow_dmeff', 'Vrel_dmeff', 'dmeff_target', 'm_dmeff'])
cosmoCDM_df = cosmoCDM_df.rename(columns={'omega_dmeff':'omega_cdm'})

# slice for faster computation. make sure you slice each dataframe the same way
cosmo_df_slice = cosmo_df[::500].reset_index(drop=True)
cosmoCDM_df_slice = cosmoCDM_df[::500].reset_index(drop=True)
df_slice = df[::500].reset_index(drop=True)

### The Viz Function

The viz function generates the entire visualizer and has three arguments:
- __data__: (DataFrame) the data you want shown as a scatter plot
- __data_classy_input__: (DataFrame) the data that you've prepared to feed into CLASS
- __data_classy_CDM__: (DataFrame) the CDM version of that data
- __class_enabled__: (Boolean) whether you want to see CLASS compute products or not 
- __latex_dict__: (dict) a dictionary with the normal param names as keys and the latex-formatted names as values

Be aware that if you pass data with categorical values (i.e. strings) in the first argument, the visualizer will break. It's fine to do so for the CLASS arguments though.

In [ ]:
# call the visualizer function
viz(df_slice, cosmo_df_slice, cosmoCDM_df_slice, class_enabled=True, latex_dict=params_latex_form)